In [ ]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.utils.data as data
import sys
from pathlib import Path

sys.path.insert(0, str( Path(Path(Path(Path(Path(__file__).parent.absolute()).parent.absolute()).parent.absolute()).parent.absolute()) ))
from Dataset import Dataset, DataLoader
from Discriminator import Discriminator
from Generator import Generator
from Loss import Loss

In [ ]:
dataset = Dataset()
data_loader = DataLoader(dataset)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

G = Generator().to(device)
D = Discriminator().to(device)
criterion = Loss()

optim_G = torch.optim.Adam(G.parameters(), lr = 0.0001)
optim_D = torch.optim.Adam(D.parameters(), lr = 0.0001)

In [ ]:
total_epochs = 200
total_batch = len(data_loader.dataloader)

for epcoh in range(total_epochs):
    avg_cost = [0, 0]
    for x, _ in data_loader.dataloader:
        real = (torch.FloatTensor(x.size(0), 1).fill_(1.0)).to(device)
        fake = (torch.FloatTensor(x.size(0), 1).fill_(0.0)).to(device)

        x = x.view(x.size(0), -1).to(device)

        noise = torch.randn(data_loader.dataloader.batch_size, 100, device = device)

        fake_img = G(noise)

        # train Generator
        optim_G.zero_grad()
        g_cost = criterion(D(fake_img), real)
        g_cost.backward()
        optim_G.step()

        fake_img = fake_img.detach().to(device) # 얘를 굳이 하는 이유가 머야

        # train Discriminator
        optim_D.zero_grad()
        d_cost = criterion(D(torch.cat((x, fake_img))), torch.cat((real, fake)))
        d_cost.backward()
        optim_D.step()

        avg_cost[0] += g_cost
        avg_cost[1] += d_cost

    avg_cost[0] /= total_batch
    avg_cost[1] /= total_batch

    if (epoch+1) % 10 == 0:
        fake_img = fake_img.reshape([batch_size, 1, 28, 28])
        img_grid = make_grid(fake_img, nrow=10, normalize=True)
        save_image(img_grid, "/content/drive/MyDrive/Deep Learning/GAN/GAN Result/%d.png"%(epoch+1))
        print("Epoch: %d, Generator: %f, Discriminator: %f"%(epoch+1, avg_cost[0], avg_cost[1]))